## Для выполнения итогового проекта требуется решить две задачи:

### 1. Построить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы.
###  2. На основе этой системы составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить  продакт-менеджеру.

### Решение первой задачи начинается в Metabase c помощью SQL-запроса, который поможет нам выяснить, какие клиенты школы купили более одного курса, то есть база данных вернет на два столбца: id таких пользователей и id курсов, покупку которых они соверишили

with a as
(
select user_id, resource_id
from final.cart_items ci 
	join final.carts c on ci.cart_id=c.id
where state='successful' and resource_type='Course'
group by 1, 2
),
b as 
(
select user_id, count(resource_id) as cc
from a
group by user_id
)
select b.user_id, a.resource_id
from b
 join a on b.user_id = a.user_id
where cc > 1
order by 1, 2

### Далее нам будет необходимо обработать данные посредством Python

In [1]:
#Загрузка необходимых библиотек
import pandas as pd
import numpy as np
import itertools
import statistics

In [2]:
#Загрузка полученного из Metabase файла с базой данный
df = pd.read_csv('finpr.csv')
df.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [3]:
#Подсчтиаем,насколько часто тот или иной курс был куплен, это даст нам понимание, какие курсы стали бестселлерами,
#а значит, их можно рекомендовать, если другие рекомендации окажутся нерелевантными из-за своей малочисленности
freq_df = df.resource_id.value_counts().reset_index()
freq_df['course_freq_perc']=freq_df.apply(lambda x: round(x['resource_id']/len(df)*100,2),axis=1)
freq_df = freq_df.rename(columns={'index':'course_id', 'resource_id':'course_freq_sum'})
freq_df

,course_id,course_freq_sum,course_freq_perc
0,551,2935,8.61
1,566,2342,6.87
2,515,1311,3.85
3,489,1125,3.30
4,490,1084,3.18
...,...,...,...
121,1182,3,0.01
122,902,3,0.01
123,1200,2,0.01
124,1199,2,0.01


In [4]:
#По условию задания, нам необходимо дать две рекомендации
#Поэтому создадим две переменные, куда впишем id двух самых популярных курсов
first_best = freq_df['course_id'].values[0]
second_best = freq_df['course_id'].values[1]


In [5]:
#Вернемся к начальному датафрейму и сгруппируем курсы относительно пользователей, совершивших покупку
pairs=df.groupby(['user_id'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
pairs.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [6]:
#Разобьем группы купленных курсов на пары
courses_list = list()
for resource in pairs['resource_id']:
    for pair in itertools.combinations (resource,2):
        courses_list.append(pair)

In [7]:
#Узнаем, сколько всего было таких пар
len(courses_list)

40017

In [8]:
#И сколько из этих пар - уникальные
courses_list_set = set(courses_list)
len(courses_list_set)


3989

In [9]:
#Подсчитаем, сколько раз была приобретена каждая пара курсов
from collections import Counter
popular= Counter ([pair for pair in courses_list])

In [10]:
#Выясним, какая пара оказалась самой популярной
{k:v for k,v in popular.items() if v == max(popular.values())}

{(551, 566): 797}

In [11]:
#Преобразуем данные счетчика в словарь с парами курсов в качестве значений и количесвом покупок в качестве ключей
sorted_pairs = {k:v for k,v in sorted(popular.items(),key=lambda item: item[0], reverse=False)}
sorted_pairs

{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

 Чтобы рекомендации отражали те пары курсов, которые встречались действительно часто и покупались многими пользователями, необходимо отделить их от того, что можно назвать случайными покупками

In [12]:
#Это можно узнать, создав спикок, элеменатми которого станут количества покупок той или иной пары курсов
values = []
for value in sorted_pairs.values():
    values.append(value)
#И вычислив персентили этого списка
statistics.quantiles(values, n=10)

[1.0, 1.0, 2.0, 2.0, 3.0, 5.0, 7.0, 11.0, 22.0]

Мы видим, что из 3989 купленных уникальных пар, подавляющее большинство(80%) было приобретено не более десяти раз. Это вполне ожидаемо, когда общее количество купленных пар равняется 40017, самая популярная пара была приобретена 797 раз, и в среднем каждая отдельная пара была куплена 10,03 раза.
Исходя из этого, пары курсов, купленные менее 11 раз, могут рассматриваться нами как случайность и не включаться в рекомендации. 

In [13]:
#Определим функцию, которая выдавать не более двух наиболее популярных парх для каждого конкретного курса 
def recommend(course):
    course_list = [] #пустой список 
    for i in sorted_pairs.keys(): #цикл проходит по ключам отсорированного словаря
        if i[0]==course and sorted_pairs[i]>=11: #если первый элемент пары курсов - id курса, поданного на вход функции и если пары курсов с ним покупались не реже порогового значения
            course_list.append((i, sorted_pairs[i])) #добавляем в пустой список кортеж и пары курсов и их количества
    sorted_course_list = sorted(course_list, key=lambda x: x[1],reverse = True) #сортировка списка по убыванию количества
    return sorted_course_list[:2] #вывод только двух значений для первой и второй рекомендаций соответственно

In [14]:
#Для дальнейщей работы возьмем только уникальные курсы, отбросив повторения
courses=df['resource_id'].unique()


### Теперь перейдем к финальной части задания - созданию таблицы

In [15]:
#Создадим датафрейм с рекомендациями, где проведем замену пустых значений на саммый полулярный и второй по популярности курс
recomend_df = pd.DataFrame (columns = ['Рекомендация №1', 'Рекомендация №2'])
for i in courses:
    if len(recommend(i)) == 2: #если функция выдает 2 рекомендации
        recomend_df.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]]
    elif len(recommend(i)) == 1: #когда рекомендация только одна
        recomend_df.loc[i] = [recommend(i)[0][0][1], first_best]
    else: #когда рекомендаций нет
        recomend_df.loc[i] = [first_best,second_best]
#Кроме того, введем в табилицу колонку с номером курса
recomend_df=recomend_df.reset_index()
recomend_df = recomend_df.rename(columns={'index':'course_id'})


In [16]:
#Посмотрим на получившийся датафрейм 
recomend_df

,course_id,Рекомендация №1,Рекомендация №2
0,516,745,553
1,1099,1139,1187
2,356,571,357
3,357,571,1125
4,1125,1186,1144
...,...,...,...
121,902,551,566
122,837,551,566
123,1200,551,566
124,833,551,566


In [17]:
#Проверим, есть ли дубликаты в строках, т.е. не совпадают ли рекомендации друг с другом
# и с курсом, к которому они выступают рекомендациями
x = pd.Series(recomend_df.apply(lambda row: row.is_unique, axis=1))
x.all()

True

### Дубликактов не обнаружено, а значит, датафрейм с рекомендациями для для продакт-менеджера и отдела маркетинга готов